In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np

In [3]:
FILE_TRAIN = 'data/training.csv'
FILE_TEST = 'data/testing.csv'

# Buscar Features en la Descripcion


## Spark

In [7]:
import pyspark
from pyspark.sql import SQLContext
import pyspark_csv as pycsv

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

sqlContext = SQLContext(sc)
sc.addPyFile('/home/marto/Desktop/pyspark_csv.py')

### Funciones

In [9]:
import re, string, unicodedata
def removePunctuation(text):
    
    text=str(text).lower().strip(' ')
    text=unicodedata.normalize('NFD', text).encode('ascii','ignore')
    text=text.decode('utf-8')
    text=re.sub(r'[^a-zA-Z0-9 ]', ' ', text)   
    return text

In [10]:
def getColums(names):
    df_index = dataframe.schema.names
    index = {}
    for name in names:
        if name in df_index:
            index[name] = df_index.index(name)
    return index

In [11]:
def letrasANumero(word):
    dic = {
        'mono': '1','uno': '1',
        'un': '1','dos': '2',
        'tres': '3','cuatro': '4',
        'cinco': '5','seis': '6',
        'siete': '7','ocho': '8'
    }
    if word in dic:
        return dic[word]
    else:
        return word

In [12]:
def wordToNumber(word):
    try:
        n = (int(letrasANumero(word)))
    except ValueError:
        return False
    return n

#### 1
Esta busqueda se encarga de buscar bigramas en donde la primera o segunda palabra sea la buscada, y la restante, un numero o una palabra que pueda ser traducida a numero

In [13]:
def buscarCantidad(string, palabra_especial, words):
    cantidad = 0
    l = string.split()
    for i in range(0,len(l)-1):
        if palabra_especial:
            if l[i] == palabra_especial[0] or l[i+1] == palabra_especial[0]:
                return palabra_especial[1]
        if l[i] in words:
            n = wordToNumber(l[i+1])
            if n and n > cantidad:
                cantidad = n
        if l[i+1] in words:
            n = wordToNumber(l[i])
            if n and n > cantidad:
                cantidad = n
    return cantidad

#### 2
Mientras que esta busqueda simplemente cuenta la cantidad de apariciones que tienen las palabras relacionadas a cada key.

In [14]:
def buscarFeature(features, des):
    d = {}
    for key in features:
        d[key] = 0
        for word in features[key]:
            if word in des:
                d[key] += 1    
    return d

La siguente funcion, busca cada uno de los valores del diccionario (mas abajo se explica y se muestra cual es) dentro de la descripcion. Existen dos tipos de busqueda distintas, (2) la busqueda de features, y (1) la busqueda de cantidad

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
def busqueda(des):
    ambientes = buscarCantidad(des,('monoambiente',1), ['ambiente','ambientes'])
    cuartos = buscarCantidad(des,False, features['cuarto'])
    banos = buscarCantidad(des,('banos',2), ['bano','banos'])
    bigram_vectorizer = CountVectorizer(ngram_range=(1, 2),token_pattern=r'\b\w+\b', min_df=1)
    analyze = bigram_vectorizer.build_analyzer()
    d = buscarFeature(features,analyze(des))
    if d['bano'] > banos:
        banos = d['bano']
    d.pop('bano', None)
    l = [ambientes,cuartos,banos] + list(d.values())
    return tuple(l)

In [16]:
def juntar(x):
    a = x[0][1]
    a_d = x[1][1]
    if (a == None) or (a > a_d):
        return tuple([x[0][0]] + list(x[1]))
    return tuple([x[0][0]] + list(x[1])[1:])

### Diccionario de featues y sinonimos:

El siguente diccionario es un diccionario de features y sinonimos, la idea de utilizar esta estructura para extraer datos de la descripcion es una solucion a los problemas obtenidos usando una lista de palabras y valores binarios para representar cada una, y luego The Hashing Trick, se logra tener el valor compacto de THT pero sin perder control sobre que es lo que buscamos y que es lo que obtenes. Tambien saber la cantidad de ciertas caracteristicas resulta util.

In [17]:
#diccionario de sinonimos
features = {
    'bano' : ['bano','banos','toilete'],
    'cuarto' : ['cuarto', 'cuartos', 'habitacion', 'habitaciones', 'dormitorio', 'dormitorios'],
    'cochera' : ['cochera','cocheras','estacionamiento','estacionamientos'],
    'seguridad' : ['guardia', 'seguridad', 'garita'],
    'gimnasio' : ['gym','gimnasio'],
    'amenities' : ['spa','laundry','solarium',''],
    'jardin' : ['parque','jardin','patio'],
    'frio' : ['split','aire acondicionado'],
    'calor' : ['caldera','losa radiante','radiadores','estufa','estufas','radiador'],
    'portero' : ['encargado', 'portero', 'porteria'],
    'pileta' : ['pileta', 'piscina'],
    'terraza' : ['terraza'],
    'balcon' : ['balcon','balcones'],
    'tipos de ambientes' : ['comedor','cocina','living','lavadero','escritorio'],
    'galeria' : ['galeria'],
    'barrio privado': ['country','barrio privado','nordelta','barrio cerrado'],
    'nuevo en venta' :['a estrenar','estrenar'],
    'ubicacion' : ['esquina','avenida','centro','esquina','plaza','frente'],
    'lujos': ['ceramicos','porcelanato','marmol','parquet','inoxidable','granito','aluminio','madera','club house'],
}

### Training Dataser

In [18]:
plaintext_rdd = sc.textFile(FILE_TRAIN)
dataframe = pycsv.csvToDataFrame(sqlContext, plaintext_rdd)
data = dataframe.rdd
columnas = ['id','description','rooms']
idx = getColums(columnas)
data = data.map(lambda x: (x[idx['id']], x[idx['rooms']], x[idx['description']]))
data = data.map(lambda x: ((x[0],x[1]),removePunctuation(x[2])))
data = data.map(lambda x: (x[0],busqueda(x[1])))
data = data.map(juntar)
columns = ['id', 'ambientes_des','cuartos'] + list(features.keys()) 
df = data.collect()
import pandas as pd
import numpy as np
df_train = pd.DataFrame(df)
df_train.columns = columns

### Testing Dataset

In [19]:
plaintext_rdd = sc.textFile(FILE_TEST)
dataframe = pycsv.csvToDataFrame(sqlContext, plaintext_rdd)
data = dataframe.rdd
columnas = ['id','description','rooms']
idx = getColums(columnas)
data = data.map(lambda x: (x[idx['id']], x[idx['rooms']], x[idx['description']]))
data = data.map(lambda x: ((x[0],x[1]),removePunctuation(x[2])))
data = data.map(lambda x: (x[0],busqueda(x[1])))
data = data.map(juntar)
columns = ['id', 'ambientes_des','cuartos'] + list(features.keys()) 
df = data.collect()
import pandas as pd
import numpy as np
df_test = pd.DataFrame(df)
df_test.columns = columns

In [20]:
sc.stop()

# Merging new data

Mientras que todos los valores conseguidos por este metodo son nuevos, la busqueda de los ambientes en la descripcion no. Es por esto que se valida si los ambientes encontrados son mayores a los que la propiedad anticipaba con anterioridad o si estos eran nulos para remplazarlos. Los demas datos se agregan al dataset original.

In [21]:
def updateRooms(row):
    a = np.isnan(row['rooms'])
    b = np.isnan(row['ambientes_des'])
    if a and not b:
        return row['ambientes_des']
    if (not a) and (not b):
        return row['ambientes_des']
    return row['rooms']

def merge_rooms(df_complete, df_rooms, FILE_NAME):
    FILE_NAME = FILE_NAME + "-with-features.csv"
    df_rooms = df_rooms.set_index(['id'])
    df_complete = df_complete.set_index(['id'])
    
    df = df_complete.join(df_rooms)
    df['rooms'] = df.apply(lambda row: updateRooms(row), axis=1)
    df = df.drop(df[['ambientes_des']], axis=1)
    df.to_csv(FILE_NAME)

In [22]:
dfT = pd.read_csv(FILE_TEST, low_memory=False)
merge_rooms(dfT,df_test, 'data/testing')

In [ ]:
dfTR = pd.read_csv(FILE_TRAIN, low_memory=False)
merge_rooms(dfTR,df_train, 'data/training')